# AbAg Binding Affinity Prediction - Colab Training

**Fast GPU training on Google Colab**

## Setup Instructions

1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → GPU (T4)
2. **Upload data**: Upload `merged_with_all_features.csv` to your Google Drive
3. **Run all cells**: Runtime → Run all

---

**Dataset Info:**
- Total samples: 390,757
- Samples with features: 330,762 (84.65%)
- Very strong binders: 230
- File size: 883 MB

**Expected Training Time:**
- 20 epochs: ~1.5-2 hours on T4 GPU
- 100 epochs: ~7-10 hours on T4 GPU

## 1. Check GPU and Install Dependencies

In [ ]:
# Check GPU availability
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Enable GPU in Runtime settings.")

In [ ]:
# Install required packages
!pip install -q transformers scikit-learn pandas numpy tqdm matplotlib seaborn
print("✅ All dependencies installed!")

## 2. Mount Google Drive and Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive mounted!")

In [ ]:
import os
from pathlib import Path

# Set up paths - MODIFY THIS to match your Google Drive location
DRIVE_DATA_PATH = "/content/drive/MyDrive/AbAg_data/merged_with_all_features.csv"
OUTPUT_DIR = "/content/drive/MyDrive/AbAg_data/models"

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Alternative: Copy data to Colab local storage for faster I/O
LOCAL_DATA_PATH = "/content/merged_with_all_features.csv"

print(f"Data path: {DRIVE_DATA_PATH}")
print(f"Output directory: {OUTPUT_DIR}")

if os.path.exists(DRIVE_DATA_PATH):
    print(f"✅ Data file found! Size: {os.path.getsize(DRIVE_DATA_PATH) / 1e6:.1f} MB")
    
    # Copy to local for faster training
    print("Copying data to local storage for faster I/O...")
    !cp "{DRIVE_DATA_PATH}" "{LOCAL_DATA_PATH}"
    print("✅ Data copied to local storage!")
    
    DATA_PATH = LOCAL_DATA_PATH
else:
    print(f"❌ Data file not found at: {DRIVE_DATA_PATH}")
    print("\nPlease upload 'merged_with_all_features.csv' to your Google Drive and update DRIVE_DATA_PATH above.")

## 3. Training Code

In [ ]:
# Imports
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr, pearsonr
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import time

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("✅ Imports complete!")

In [ ]:
# Dataset class
class AffinityDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

print("✅ Dataset class defined!")

In [ ]:
# Model architecture
class AffinityPredictor(nn.Module):
    def __init__(self, input_dim=150, hidden_dims=[256, 128], dropout=0.3):
        super(AffinityPredictor, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x).squeeze()

print("✅ Model class defined!")

In [ ]:
# Weighted MSE Loss
class WeightedMSELoss(nn.Module):
    def __init__(self, bin_weights, bins_edges):
        super().__init__()
        self.bin_weights = bin_weights
        self.bins = bins_edges
    
    def forward(self, predictions, targets):
        # Calculate weights for each sample
        weights = torch.ones_like(targets)
        for i, (low, high) in enumerate(zip(self.bins[:-1], self.bins[1:])):
            mask = (targets >= low) & (targets < high)
            weights[mask] = self.bin_weights[i]
        
        # Weighted MSE
        mse = (predictions - targets) ** 2
        weighted_mse = mse * weights
        return weighted_mse.mean()

print("✅ Weighted MSE loss defined!")

## 4. Load and Prepare Data

In [ ]:
print("Loading dataset...")
df = pd.read_csv(DATA_PATH, low_memory=False)
print(f"✅ Loaded {len(df):,} samples")

# Filter samples with features
pca_cols = [f'esm2_pca_{i}' for i in range(150)]
df_with_features = df[df[pca_cols[0]].notna()].copy()
print(f"✅ Samples with features: {len(df_with_features):,}")

# Create affinity bins
BINS = [0, 5, 7, 9, 11, 16]
BIN_LABELS = ['very_weak', 'weak', 'moderate', 'strong', 'very_strong']
df_with_features['affinity_bin'] = pd.cut(
    df_with_features['pKd'], bins=BINS, labels=BIN_LABELS, include_lowest=True
)

# Show distribution
print("\nAffinity Distribution:")
for label in BIN_LABELS:
    count = (df_with_features['affinity_bin'] == label).sum()
    pct = count / len(df_with_features) * 100
    print(f"  {label:<15}: {count:6,} ({pct:5.2f}%)")

print(f"\nTotal: {len(df_with_features):,}")

In [ ]:
# Extract features and labels
X = df_with_features[pca_cols].values
y = df_with_features['pKd'].values

# Train/val/test split
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.15/0.85, random_state=42)

print(f"Train set: {len(X_train):,}")
print(f"Val set: {len(X_val):,}")
print(f"Test set: {len(X_test):,}")

# Calculate class weights
y_train_binned = pd.cut(y_train, bins=BINS, labels=BIN_LABELS, include_lowest=True)
bin_counts = y_train_binned.value_counts().sort_index()
total_samples = len(y_train)
bin_weights = {}
for label in BIN_LABELS:
    count = bin_counts.get(label, 1)
    bin_weights[label] = total_samples / (len(BIN_LABELS) * count)

print("\nClass weights:")
for label, weight in bin_weights.items():
    print(f"  {label:<15}: {weight:.2f}")

# Convert to tensor
bin_weights_tensor = torch.FloatTensor([bin_weights[l] for l in BIN_LABELS]).cuda()

# Create datasets
train_dataset = AffinityDataset(X_train, y_train)
val_dataset = AffinityDataset(X_val, y_val)
test_dataset = AffinityDataset(X_test, y_test)

print("\n✅ Data preparation complete!")

## 5. Training

In [ ]:
# Training configuration
EPOCHS = 100  # Change to 20 for quick test
BATCH_SIZE = 128  # Can use larger batch size on Colab
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

print(f"Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Device: {DEVICE}")

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print(f"✅ Data loaders created!")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")

In [ ]:
# Initialize model
model = AffinityPredictor(input_dim=150, hidden_dims=[256, 128], dropout=0.3)
model = model.to(DEVICE)

# Loss and optimizer
criterion = WeightedMSELoss(bin_weights_tensor, BINS)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✅ Model initialized!")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"\nModel architecture:")
print(model)

In [ ]:
# Training loop
best_val_loss = float('inf')
train_losses = []
val_losses = []
train_start = time.time()

for epoch in range(EPOCHS):
    epoch_start = time.time()
    
    # Training
    model.train()
    train_loss = 0
    train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")
    
    for features, labels in train_pbar:
        features, labels = features.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        predictions = model(features)
        loss = criterion(predictions, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        train_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    train_loss /= len(train_loader)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    val_loss = 0
    val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Val]")
    
    with torch.no_grad():
        for features, labels in val_pbar:
            features, labels = features.to(DEVICE), labels.to(DEVICE)
            predictions = model(features)
            loss = criterion(predictions, labels)
            val_loss += loss.item()
            val_pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    val_loss /= len(val_loader)
    val_losses.append(val_loss)
    
    # Update learning rate
    scheduler.step(val_loss)
    
    epoch_time = time.time() - epoch_start
    
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time: {epoch_time:.1f}s")
    
    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, f'{OUTPUT_DIR}/best_model.pth')
        print(f"  ✅ New best model saved! (val_loss: {val_loss:.4f})")
    
    # Save checkpoint every 10 epochs
    if (epoch + 1) % 10 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, f'{OUTPUT_DIR}/checkpoint_epoch_{epoch+1}.pth')
        print(f"  💾 Checkpoint saved!")

total_time = time.time() - train_start
print(f"\n✅ Training complete! Total time: {total_time/3600:.2f} hours")

## 6. Evaluation

In [ ]:
# Plot training curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.savefig(f'{OUTPUT_DIR}/training_curves.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Training curves saved!")

In [ ]:
# Load best model
checkpoint = torch.load(f'{OUTPUT_DIR}/best_model.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Best model loaded from epoch {checkpoint['epoch']+1}")
print(f"   Val loss: {checkpoint['val_loss']:.4f}")

In [ ]:
# Evaluate on test set
model.eval()
test_predictions = []
test_targets = []

with torch.no_grad():
    for features, labels in tqdm(test_loader, desc="Testing"):
        features = features.to(DEVICE)
        predictions = model(features)
        test_predictions.extend(predictions.cpu().numpy())
        test_targets.extend(labels.numpy())

test_predictions = np.array(test_predictions)
test_targets = np.array(test_targets)

# Calculate metrics
mse = mean_squared_error(test_targets, test_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(test_targets, test_predictions)
spearman = spearmanr(test_targets, test_predictions)[0]
pearson = pearsonr(test_targets, test_predictions)[0]
r2 = 1 - (np.sum((test_targets - test_predictions)**2) / np.sum((test_targets - test_targets.mean())**2))

print("="*60)
print("TEST SET PERFORMANCE")
print("="*60)
print(f"RMSE:        {rmse:.4f}")
print(f"MAE:         {mae:.4f}")
print(f"Spearman ρ:  {spearman:.4f}")
print(f"Pearson r:   {pearson:.4f}")
print(f"R²:          {r2:.4f}")
print("="*60)

In [ ]:
# Per-bin metrics
test_df = pd.DataFrame({
    'target': test_targets,
    'prediction': test_predictions
})
test_df['affinity_bin'] = pd.cut(test_df['target'], bins=BINS, labels=BIN_LABELS, include_lowest=True)

print("\nPER-BIN PERFORMANCE:")
print("="*60)
print(f"{'Bin':<15} | {'Count':<8} | {'RMSE':<8} | {'MAE':<8}")
print("-"*60)

for label in BIN_LABELS:
    bin_data = test_df[test_df['affinity_bin'] == label]
    if len(bin_data) > 0:
        bin_rmse = np.sqrt(mean_squared_error(bin_data['target'], bin_data['prediction']))
        bin_mae = mean_absolute_error(bin_data['target'], bin_data['prediction'])
        print(f"{label:<15} | {len(bin_data):<8} | {bin_rmse:<8.4f} | {bin_mae:<8.4f}")

print("="*60)

In [ ]:
# Plot predictions vs targets
plt.figure(figsize=(10, 10))
plt.scatter(test_targets, test_predictions, alpha=0.3, s=10)
plt.plot([test_targets.min(), test_targets.max()], [test_targets.min(), test_targets.max()], 'r--', lw=2)
plt.xlabel('True pKd')
plt.ylabel('Predicted pKd')
plt.title(f'Test Set Predictions (Spearman ρ = {spearman:.4f}, RMSE = {rmse:.4f})')
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.savefig(f'{OUTPUT_DIR}/predictions_vs_targets.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Prediction plot saved!")

In [ ]:
# Residuals plot
residuals = test_predictions - test_targets

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Residuals vs predictions
axes[0].scatter(test_predictions, residuals, alpha=0.3, s=10)
axes[0].axhline(y=0, color='r', linestyle='--', lw=2)
axes[0].set_xlabel('Predicted pKd')
axes[0].set_ylabel('Residuals (Predicted - True)')
axes[0].set_title('Residuals vs Predictions')
axes[0].grid(True, alpha=0.3)

# Residuals distribution
axes[1].hist(residuals, bins=50, edgecolor='black')
axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Residuals')
axes[1].set_ylabel('Frequency')
axes[1].set_title(f'Residuals Distribution (Mean = {residuals.mean():.4f}, Std = {residuals.std():.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{OUTPUT_DIR}/residuals_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print("✅ Residuals analysis saved!")

## 7. Save Results

In [ ]:
# Save metrics to file
metrics_summary = f"""
AbAg Binding Affinity Prediction - Training Results
{'='*60}

Training Configuration:
  - Epochs: {EPOCHS}
  - Batch size: {BATCH_SIZE}
  - Learning rate: {LEARNING_RATE}
  - Training samples: {len(X_train):,}
  - Validation samples: {len(X_val):,}
  - Test samples: {len(X_test):,}
  - Total training time: {total_time/3600:.2f} hours

Test Set Performance:
  - RMSE: {rmse:.4f}
  - MAE: {mae:.4f}
  - Spearman ρ: {spearman:.4f}
  - Pearson r: {pearson:.4f}
  - R²: {r2:.4f}

Per-Bin Performance:
"""

for label in BIN_LABELS:
    bin_data = test_df[test_df['affinity_bin'] == label]
    if len(bin_data) > 0:
        bin_rmse = np.sqrt(mean_squared_error(bin_data['target'], bin_data['prediction']))
        bin_mae = mean_absolute_error(bin_data['target'], bin_data['prediction'])
        metrics_summary += f"  - {label}: RMSE={bin_rmse:.4f}, MAE={bin_mae:.4f}, N={len(bin_data)}\n"

metrics_summary += f"\n{'='*60}\n"

# Save to file
with open(f'{OUTPUT_DIR}/training_results.txt', 'w') as f:
    f.write(metrics_summary)

print(metrics_summary)
print(f"✅ Results saved to {OUTPUT_DIR}/training_results.txt")

In [ ]:
# Save final model with metadata
torch.save({
    'model_state_dict': model.state_dict(),
    'metrics': {
        'rmse': rmse,
        'mae': mae,
        'spearman': spearman,
        'pearson': pearson,
        'r2': r2
    },
    'config': {
        'input_dim': 150,
        'hidden_dims': [256, 128],
        'dropout': 0.3,
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'learning_rate': LEARNING_RATE
    }
}, f'{OUTPUT_DIR}/final_model.pth')

print(f"✅ Final model saved to {OUTPUT_DIR}/final_model.pth")
print(f"\nAll files saved to: {OUTPUT_DIR}")
print("\nYou can now download the trained model from Google Drive!")